In [1]:
##LOCAL TEST

import scripts.preprocess as pp
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

master, payments = pp.LoadData()
print('Data Loaded')
solution, master = pp.PrepMLPData(master, payments)
print('Solution String Created')
train_dl, test_dl = pp.CreateMLPDataLoader(master, solution)
print('Data Loader Created')
filename = os.path.join(data_path, 'train_dl.pt')
torch.save(train_dl, 'train_dl.pt')
filename = os.path.join(data_path, 'test_dl.pt')
torch.save(test_dl, 'test_dl.pt')
print('YAY!')

Data Loaded
Data Prepped
Data Loader Created
YAY!
Data Loaded
Solution String Created
Data Loader Created


NameError: name 'data_path' is not defined

In [1]:
import azurescript as azs

azs.ConnectToAzure()
# azs.UploadData()
azs.CreateAzureDataset()
azs.PrepareAzureScript()
azs.RunAzureScript()
print('done')

Workspace: AzureML-Pytorch
Compute Target: AzPytrch-NC6
Datastore: workspaceblobstore
Experiment: ANFIS-PyTorch
[({
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-37cc9333-cffe-4a2a-ba89-7ed5c59dd573",
  "account_name": "azuremlpytorch4611169748",
  "protocol": "https",
  "endpoint": "core.windows.net"
}, 'data')]


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

done
